#### TODO
- Check how to get around article limit of 10000 if possible
- Can use history server for other purposes?
- What can be the advantage for using eutilities VS edirect / usecases?

In [25]:
import os
print(os.environ.get('REQUESTS_CA_BUNDLE'))
print(os.environ.get('SSL_CERT_FILE'))


None
C:\Users\Gebruiker\anaconda3\Library\ssl\cacert.pem


In [30]:
from Bio import Entrez
import time

In [31]:
def build_query():
    searchterm = input("Enter query search term: ")
    article_type = ('[IT] AND ("Clinical Trial"[PT] OR "Randomized Controlled Trial"[PT] OR "Meta-Analysis"[PT] '
                    'OR "Systematic Review"[PT] OR "Comparative Study"[PT] OR "Observational Study"[PT] '
                    'OR "Validation Study"[PT] OR "Review"[PT])')
    
    full_query = searchterm + article_type
    return full_query


In [32]:
def history(full_query):
    API_key = input("Enter API_key:")

    # Get number of data records
    with Entrez.esearch(db="pubmed", term=full_query, retmax=1, api_key=API_key, usehistory="y") as handle:
        results = Entrez.read(handle)
        count = int(results["Count"])
        webenv = results["WebEnv"]    
        query_key = results["QueryKey"]
    
    print(f"Total articles found: {count}")
    print(f"WebEnv: {webenv}, QueryKey: {query_key}")
    
    return webenv, query_key, count, API_key


In [19]:
def fetch_data(webenv, query_key, count, API_key, batchsize=5000):
    article_data = []
    max_records = min(count, 9999)

    for start in range(0, max_records, batchsize):
        print(f"Records {start + 1} to {min(start + batchsize, count)}...")
        
        # Request data from PubMed using efetch, and specify 'json' for retmode
        with Entrez.efetch(db="pubmed", query_key=query_key, webenv=webenv, retstart=start, retmax=batchsize, 
                           rettype="json", api_key=API_key) as handle:
            data = handle.read()
            article_data.append(data)
            
            # Save data to a JSON file
            filename = f"pubmed_data_{start+1}_{min(start + batchsize, count)}.json"
            with open(filename, 'w') as file:
                file.write(data.decode("utf-8"))
        
        time.sleep(10)

    print("Data fetching complete.")
    return article_data

In [33]:
Entrez.email = input("Enter emailadres:")
query = build_query()
webenv, query_key, count, API_key = history(query)
articles = fetch_data(webenv, query_key, count, API_key)

Enter emailadres: michael.viroux@gmail.com
Enter query search term:  "vitamin d"
Enter API_key: 6898e4b9cc65f86c658e0ffffad70d6c0209


Total articles found: 28741
WebEnv: MCID_67af86c007a5a5787002475a, QueryKey: 1


NameError: name 'fetch_data' is not defined

In [22]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [23]:
!pip install pandas

  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ------ --------------------------------- 1.8/11.5 MB 9.1 MB/s eta 0:00:02
   ------------ --------------------------- 3.7/11.5 MB 9.5 MB/s eta 0:00:01
   -------------------- ------------------- 6.0/11.5 MB 10.0 MB/s eta 0:00:01
   --------------------------------- ------ 9.7/11.5 MB 12.1 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 12.0 MB/s eta 0:00:00
Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)


In [27]:
import pandas as pd
!pip install lxml

   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   --------------------------- ------------ 2.6/3.8 MB 12.5 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 10.8 MB/s eta 0:00:00


In [5]:
"""
    To retrieve more than 10,000 UIDs from databases other than PubMed, 
    submit multiple esearch requests while incrementing the value of retstart (see Application 3). 
    For PubMed, ESearch can only retrieve the first 10,000 records matching the query. 
    To obtain more than 10,000 PubMed records, consider using <EDirect> that contains additional logic
    to batch PubMed search results automatically so that an arbitrary number can be retrieved.
    For details see https://www.ncbi.nlm.nih.gov/books/NBK25499/
"""

'\n    To retrieve more than 10,000 UIDs from databases other than PubMed, \n    submit multiple esearch requests while incrementing the value of retstart (see Application 3). \n    For PubMed, ESearch can only retrieve the first 10,000 records matching the query. \n    To obtain more than 10,000 PubMed records, consider using <EDirect> that contains additional logic\n    to batch PubMed search results automatically so that an arbitrary number can be retrieved.\n    For details see https://www.ncbi.nlm.nih.gov/books/NBK25499/\n'

In [6]:
from Bio import Entrez
import time
import pandas as pd
import json

def build_query():
    searchterm = input("Enter query search term: ")
    article_type = ('[IT] AND ("Clinical Trial"[PT] OR "Randomized Controlled Trial"[PT] OR "Meta-Analysis"[PT] '
                    'OR "Systematic Review"[PT] OR "Comparative Study"[PT] OR "Observational Study"[PT] '
                    'OR "Validation Study"[PT] OR "Case Reports"[PT] OR "Review"[PT])')

    full_query = searchterm + article_type
    return full_query


def history(full_query):
    API_key = input("Enter API_key:")

    # Get number of data records
    with Entrez.esearch(db="pubmed", term=query, retmax=1, api_key=API_key, usehistory="y") as handle:
        results = Entrez.read(handle)
        count = int(results["Count"])
        webenv = results["WebEnv"]
        query_key = results["QueryKey"]

    print(f"Total articles found: {count}")
    print(f"WebEnv: {webenv}, QueryKey: {query_key}")

    return webenv, query_key, count, API_key


def fetch_data(webenv, query_key, count, API_key, batchsize=5000):
    max_records = min(count, 9999)
    df = pd.DataFrame(columns=['PMID', 'Title', 'Abstract', 'Authors', 'Journal', 'Keywords', 'URL', 'Affiliations'])

    for start in range(0, max_records, batchsize):
        print(f"Records {start + 1} to {min(start + batchsize, count)}...")

        # Request data from PubMed using efetch, and specify 'json' for retmode
        with Entrez.efetch(db="pubmed", query_key=query_key, webenv=webenv, retstart=start, retmax=batchsize,
                           retmode="xml", api_key=API_key) as handle:
            data = Entrez.read(handle)

            # Process each article
            for record in data.get('PubmedArticle', []):
                pmid = record['MedlineCitation']['PMID']
                title = record['MedlineCitation']['Article']['ArticleTitle']

                abstract = ' '.join(record['MedlineCitation']['Article']['Abstract']['AbstractText']) if 'Abstract' in \
                                    record['MedlineCitation']['Article'] and 'AbstractText' in \
                                    record['MedlineCitation']['Article']['Abstract'] else ''

                authors = ', '.join(
                    f"{author.get('LastName', '')} {author.get('ForeName', '')}".strip()
                    for author in record['MedlineCitation']['Article'].get('AuthorList', [])
                )

                affiliations = []
                for author in record['MedlineCitation']['Article'].get('AuthorList', []):
                    if 'AffiliationInfo' in author and author['AffiliationInfo']:
                        affiliations.append(author['AffiliationInfo'][0]['Affiliation'])
                affiliations = '; '.join(set(affiliations))

                journal = record['MedlineCitation']['Article']['Journal']['Title']
                keywords = ', '.join(
                    keyword['DescriptorName'] for keyword in record['MedlineCitation'].get('MeshHeadingList', [])
                )
                url = f"https://www.ncbi.nlm.nih.gov/pubmed/{pmid}"

                # Append the data to the DataFrame
                df.loc[len(df)] = [pmid, title, abstract, authors, journal, keywords, url, affiliations]

        time.sleep(10)

    print("Data fetching complete.")
    return df

Entrez.email = input("Enter emailadres:")
query = build_query()
webenv, query_key, count, API_key = history(query)


if count > 0:
    df = fetch_data(webenv, query_key, count, API_key)
    print(df.head())  # Display a preview of the data
    df.to_csv("pubmed_results.csv", index=False)  # Save results to a CSV file
    print("Data saved to pubmed_results.csv")
else:
    print("No articles found.")

Enter emailadres: michael.viroux@gmail.com
Enter query search term:  "vitamin d"
Enter API_key: 6898e4b9cc65f86c658e0ffffad70d6c0209


Total articles found: 32688
WebEnv: MCID_67ac943530c0976af50e07f5, QueryKey: 1
Records 1 to 5000...
Records 5001 to 10000...
Data fetching complete.
       PMID                                              Title  \
0  39934684  Association of aberrant mineral metabolic mark...   
1  39934504  [Nonoperative treatment for the management of ...   
2  39931432  Shining light on knee osteoarthritis: an overv...   
3  39930022  Dietary supplements and prevention of preeclam...   
4  39925524  Traumatic Anserine Folliculosis: A Case Report...   

                                            Abstract  \
0  This meta-analysis aims to investigate the imp...   
1  Today, there are several nonoperative treatmen...   
2  The impact of knee osteoarthritis on individua...   
3  Preeclampsia (PE) is a common pregnancy compli...   
4  Traumatic anserine folliculosis (TAF) is a les...   

                                             Authors  \
0  Liu Yao, Zhang Zhen Xing, Fu Chen Sheng, Ye Zh...   
1  Ho

In [32]:
import pandas as pd
import json
from Bio import Entrez

# Set the email address to avoid any potential issues with Entrez
Entrez.email = 'michael.viroux@gmail.com'

# Define lists of authors and topics
authors = [] #['Bryan Holland', 'Mehmet Oz', 'Anthony Fauci']  # Example authors, adjust as needed
topics = ['RNA', 'cardiovascular']  # Example topics, adjust as needed

# Define date range
date_range = '("2012/03/01"[Date - Create] : "2022/12/31"[Date - Create])'

# Build the query dynamically based on the available authors and topics
queries = []

if authors:
    author_queries = ['{}[Author]'.format(author) for author in authors]
    queries.append('(' + ' OR '.join(author_queries) + ')')

if topics:
    topic_queries = ['{}[Title/Abstract]'.format(topic) for topic in topics]
    queries.append('(' + ' OR '.join(topic_queries) + ')')

full_query = ' AND '.join(queries) + ' AND ' + date_range

# Search PubMed for relevant records
handle = Entrez.esearch(db='pubmed', retmax=11, term=full_query)
record = Entrez.read(handle)
id_list = record['IdList']

# DataFrame to store the extracted data
df = pd.DataFrame(columns=['PMID', 'Title', 'Abstract', 'Authors', 'Journal', 'Keywords', 'URL', 'Affiliations'])

# Fetch information for each record in the id_list
for pmid in id_list:
    handle = Entrez.efetch(db='pubmed', id=pmid, retmode='xml')
    records = Entrez.read(handle)

    # Process each PubMed article in the response
    for record in records['PubmedArticle']:
        # Print the record in a formatted JSON style
        print(json.dumps(record, indent=4, default=str))  # default=str handles types JSON can't serialize like datetime
        title = record['MedlineCitation']['Article']['ArticleTitle']
        abstract = ' '.join(record['MedlineCitation']['Article']['Abstract']['AbstractText']) if 'Abstract' in record['MedlineCitation']['Article'] and 'AbstractText' in record['MedlineCitation']['Article']['Abstract'] else ''
        authors = ', '.join(author.get('LastName', '') + ' ' + author.get('ForeName', '') for author in record['MedlineCitation']['Article']['AuthorList'])
        
        affiliations = []
        for author in record['MedlineCitation']['Article']['AuthorList']:
            if 'AffiliationInfo' in author and author['AffiliationInfo']:
                affiliations.append(author['AffiliationInfo'][0]['Affiliation'])
        affiliations = '; '.join(set(affiliations))

        journal = record['MedlineCitation']['Article']['Journal']['Title']
        keywords = ', '.join(keyword['DescriptorName'] for keyword in record['MedlineCitation']['MeshHeadingList']) if 'MeshHeadingList' in record['MedlineCitation'] else ''
        url = f"https://www.ncbi.nlm.nih.gov/pubmed/{pmid}"

        new_row = pd.DataFrame({
            'PMID': [pmid],
            'Title': [title],
            'Abstract': [abstract],
            'Authors': [authors],
            'Journal': [journal],
            'Keywords': [keywords],
            'URL': [url],
            'Affiliations': [affiliations]
        })

        df = pd.concat([df, new_row], ignore_index=True)

# Save DataFrame to an Excel file
df.to_csv('PubMed_resultsx.csv', index=False)

{
    "MedlineCitation": {
        "CitationSubset": [
            "IM"
        ],
        "OtherAbstract": [],
        "InvestigatorList": [],
        "OtherID": [],
        "KeywordList": [
            [
                "B cells",
                "SARS-CoV-2",
                "adaptive immunity",
                "antibodies",
                "mRNA vaccines"
            ]
        ],
        "SpaceFlightMission": [],
        "GeneralNote": [],
        "PMID": "35759653",
        "DateCompleted": {
            "Year": "2022",
            "Month": "06",
            "Day": "29"
        },
        "DateRevised": {
            "Year": "2024",
            "Month": "12",
            "Day": "05"
        },
        "Article": {
            "ArticleDate": [
                {
                    "Year": "2022",
                    "Month": "06",
                    "Day": "27"
                }
            ],
            "ELocationID": [
                "e2204607119",
                "10.1073/pna